In [ ]:
import pandas as pd
import psycopg2

pd.options.display.max_rows = 500
pd.options.display.max_columns = 30

In [ ]:
COUNTRY = 'netherlands'

In [ ]:
from utilities import db_to_pandas

sql_query = """
SELECT tweet_id, concepts, tweet_body->>'created_at' as created_at
FROM tweet
WHERE location_id LIKE '{}%'
AND concepts IS NOT NULL
AND tweet_body->>'lang' = 'nl'
;
""".format(COUNTRY)

df_from_db = db_to_pandas(sql_query)

In [ ]:
# convert to timestamps to days
df_from_db['date'] = pd.to_datetime(df_from_db['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')
df_from_db['date'] = df_from_db.date.dt.normalize()

df_from_db.head(10)

In [ ]:
# expand concepts to have its own series
a = df_from_db.explode('concepts')
concept_df = (a.assign(concepts = a['concepts'].str.get('surfaceForm'))\
              .groupby(['date','concepts'])['concepts'].count().reset_index(name='count'))
concept_df

In [ ]:
top_concepts = concept_df.groupby(['concepts']).agg({'count':sum}).sort_values(['count'], ascending=False)
top_concepts

In [ ]:
# find interesting concepts to take into account
top_concepts.head(200)

In [ ]:
stop = ('coronavirus', 'Coronavirus', 'Corona', 'CoronaVirus', 'do', 'amp', 'Deutschland', 'via', 'tun', 'primer', 'un', 'del')
relevant_top = concept_df[~concept_df.concepts.isin(stop)]
relevant_top.groupby(['concepts']).agg({'count':sum}).sort_values(['count'], ascending=False)

In [ ]:
def merge_topics(df, topic):
    cdf = df[df['concepts'].isin(topic['topics'])]
    cdf.loc[:,'concepts'] = topic['name']
    return cdf.groupby(['date']).agg({'count':sum}).reset_index()

In [ ]:
if COUNTRY == 'germany':
    protect = {'name': 'protection','topics': ['Desinfektionsmittel', 'Quarantäne', 'Mundschutz', 'Klopapier', 
                                               'Hände schütteln', 'Hygiene']}
    countries = {'name': 'other countries', 'topics': ['EU', 'Italien', 'Mailand', 'Hongkong', 'Israel', 'Roma', 'Iran', 'Südkorea', 'China', 'china', 
                                                       'US', 'USA', 'Geneva', 'Österreich']}
    sources = {'name': 'national institutions', 'topics': ['Virologe', 'Robert-Koch Institut', 'RKI', 
                                                         'Jens Spahn', 'Bundesregierung', 'Merkel', 
                                                         'Angela Merkel', 'Bundesgesundheitsminister']}
elif COUNTRY == 'netherlands':
    protect = {'name': 'protection','topics': ['quarantaine', 'volksgezondheid', 'vaccin', 'supermarkt']}
    countries = {'name': 'other countries', 'topics': ['EU', 'Iran', 'China', 'china', 'Italiaanse', 'Italië', 'Korea', 
                                                       'Japan', 'Duitsland', 'Frankrijk', 'Israël', 'Lombardije', 'Zuid-Korea', 'België']}
    sources = {'name': 'national institutions', 'topics': ['RIVM', 'GGD', 'rivm', 'vws', 'VWS', 'Mark Rutte', 'Viroloog']}
else:
    print("invalid COUNTRY")


df1 = merge_topics(concept_df, protect)
df2 = merge_topics(concept_df, countries)
df3 = merge_topics(concept_df, sources)
df1['concepts'] = 'protection'
df2['concepts'] = 'other countries'
df3['concepts'] = 'national institutions'
interesting_df = pd.concat([df1, df2, df3])
interesting_df.head()

In [ ]:
sql_query = """
SELECT tweet_body->>'created_at' as created_at
FROM tweet
WHERE location_id LIKE '{}%'
AND tweet_body->>'lang' = 'nl'
;
""".format(COUNTRY)

all_tweets = db_to_pandas(sql_query)
all_tweets['date'] = pd.to_datetime(all_tweets['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')
all_tweets['date'] = all_tweets.date.dt.normalize()

total_tweets = all_tweets['date'].value_counts()
total_tweets = total_tweets.to_frame().reset_index().rename(columns={'index': 'date', 'date': 'count'})
total_tweets.info
#total_tweets = total_tweets.reset_index()


total_tweets

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import rcParams

sns.set(style="dark")
palette = sns.color_palette("bright", 3)
plt.title("DBPedia concepts identified in tweets in {}".format(COUNTRY.title()))

#sns.set_context("poster")
plt.rcParams.update({'font.size': 30, 'figure.figsize': (8, 4) })

                         
# first axis
plot_df = interesting_df

print(plot_df.head(10))
ax1 = sns.lineplot(data=plot_df, x='date', y='count', hue='concepts', linewidth=2, palette=palette)

# second axis
ax2 = plt.twinx()
sns.lineplot(data=total_tweets, x='date', y='count', color="grey", ax=ax2, linewidth=2)
ax2.grid(False)

ax1.set(ylabel='Number of concepts in tweet text', xlabel=None)

ax2.set(ylabel='Total tweets')
ax2.lines[0].set_linestyle("--")

plt.legend(labels=['Total tweets', 'x'])

plt.gcf().autofmt_xdate()


plt.savefig('concepts_over_time_{}.png'.format(COUNTRY))